<a href="https://colab.research.google.com/github/PochuanChen/JamesGo/blob/main/James_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [1]:
import numpy
import matplotlib.pyplot as plt
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import time
import random
from copy import deepcopy
from collections import defaultdict
import sys
import json
import tensorflow as tf
from tensorflow import keras
from os.path import exists
from pathlib import Path

In [2]:
!pip install -q ipycanvas==0.11
from google.colab import output
output.enable_custom_widget_manager()

from ipycanvas import Canvas, hold_canvas
from ipywidgets import Output





     |████████████████████████████████| 255 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 61.6 MB/s 


# Game

In [3]:


class Game():
  def __init__(self, size):
    self.size = size
    #self.board = numpy.zeros((size, size), dtype=int)
    self.board = [[0 for i in range(size)] for j in range(size)]
    self.show_board = self.board
    self.group_ = [[None for i in range(size)] for j in range(size)]
    self.turn = 1
    self.__pass = 0
    self.step = 0
    self.history = {"move":[None], "board":[deepcopy(self.board)]}
    self.last_move = None
    self.__legal_moves = None #private?
    self.Ko = None
    self.state = "Proceeding"

  def regroup_all(self):
    self.group_ = [[None for i in range(self.size)] for j in range(self.size)]
    for x in range(self.size):
      for y in range(self.size):
        if self.board[x][y] == self.turn:
          self.group([x,y])


    self.turn = -self.turn
    for x in range(self.size):
      for y in range(self.size):
        if self.board[x][y] == self.turn:
          self.group([x,y])

    for x in range(self.size):
      for y in range(self.size):
        if self.board[x][y] == 0:
          self.check_eye([x,y])

    self.turn = -self.turn

    for x in range(self.size):
      for y in range(self.size):
        if self.board[x][y] == 0:
          self.check_eye([x,y])
    return

  def group(self, move):
    x, y = move
    groupingA = []
    groupingB = []
    qi = set()
    
    if x>0:
      g = self.group_[x-1][y]
      if g:
        if self.board[x-1][y] == self.turn:
          groupingA.append(g)
        elif self.board[x-1][y] == -self.turn:
          groupingB.append(g)
      else:
        qi.add((x-1,y))

    if x<len(self.group_)-1:
      g = self.group_[x+1][y]
      if g:
        if self.board[x+1][y] == self.turn:
          groupingA.append(g)
        elif self.board[x+1][y] == -self.turn:
          groupingB.append(g)
      else:
        qi.add((x+1,y))

    if y>0:
      g = self.group_[x][y-1]
      if g:
        if self.board[x][y-1] == self.turn:
          groupingA.append(g)
        elif self.board[x][y-1] == -self.turn:
          groupingB.append(g)
      else:
        qi.add((x,y-1))

    if y<len(self.group_[0])-1:
      g = self.group_[x][y+1]
      if g:
        if self.board[x][y+1] == self.turn:
          groupingA.append(g)
        elif self.board[x][y+1] == -self.turn:
          groupingB.append(g)
      else:
        qi.add((x,y+1))

    if not groupingA:
      self.group_[x][y] = {"player": self.turn, "stone": [move], "qi": qi, "eye": set(), "ey":set()}
    else:
      g = groupingA.pop()
      g["stone"].append(move)
      g["qi"].discard(tuple(move))
      g["qi"] |= qi
      g["eye"].discard(tuple(move))
      g["ey"].discard(tuple(move))
      self.group_[x][y] = g
      for l in groupingA:
        if l != g:
          l["qi"].discard(tuple(move))
          l["eye"].discard(tuple(move))
          l["ey"].discard(tuple(move))
          g["stone"].extend(l["stone"])
          g["qi"] |= l["qi"]
          g["eye"] |= l["eye"]
          g["ey"] |= l["ey"]
          for x,y in l["stone"]:
            self.group_[x][y] = g

    if groupingB:
      g = groupingB.pop()
      g["qi"].discard(tuple(move))
      for l in groupingB:
        if l != g:
          l["qi"].discard(tuple(move))

    return

  def make_move(self, move):
    self.show_board = self.board
    self.step = len(self.history["board"])-1

    if self.state == "Over":
      return self.check_Win()

    if self.__legal_moves == None:
      self.moves()

    if move != "Pass" and not move in self.__legal_moves:
      print (move, " is not a legal move!")
      print (self.__legal_moves)
      assert False, "illegal move"

    # if a game lasts too long or game result can be judged, stop the game.
    black,white = self.check_Land()
    #total = black+white
    if self.step > 3*self.size*self.size or\
      black > 0.5*self.size*self.size or\
      white > 0.5*self.size*self.size:
      self.state = "Over"
      self.step += 1
      if isinstance(move, list):
        self.board[move[0]][move[1]] = self.turn
        self.group(move)
        self.history["move"].append([move[0],move[1],0])
        self.check_Kill(move)
        self.__pass = 0
      elif move == "Pass":
        self.__pass += 1
        self.history["move"].append([move,0])
      else:
        assert False

      self.history["board"].append(deepcopy(self.board))
      self.__legal_moves = None
      self.turn = -self.turn
      self.regroup_all() #to check new eyes
      black,white = self.check_Land()
      winner = self.check_Win()
      self.history["result"] = [winner, black, white, "Judge"]        
      return winner

    if move == "Pass":
      self.__pass += 1
      if self.__pass >=2:
        self.state = "Over"
        self.step += 1
        self.history["board"].append(deepcopy(self.board))
        self.history["move"].append([move,0])
        self.__legal_moves = None
        self.turn = -self.turn
        black,white = self.check_Land()
        winner = self.check_Win()
        self.history["result"] = [winner, black, white]        
        return winner
    else:
      self.board[move[0]][move[1]] = self.turn
      self.group(move)
      if self.check_Kill(move) == 1: self.check_Ko(move)
      self.__pass = 0

    self.step += 1
    self.history["board"].append(deepcopy(self.board))

    if isinstance(move,list):
      move_ko = [move[0], move[1], 0]
      if self.Ko:
        move_ko.append([self.Ko[0], self.Ko[1]])
      self.history["move"].append(move_ko)
    else:
      self.history["move"].append([move,0])
    self.__legal_moves = None
    self.turn = -self.turn

    return


  def unmake_move(self):
    #assert self.step > 0, "unmake_move and step <=0"
    if self.step > 0:
      if self.step % 2: # considering undo game in history
        self.turn = 1
      else:
        self.turn = -1
      self.step -= 1
      self.history["board"].pop()
      self.board = deepcopy(self.history["board"][-1])
      move = self.history["move"].pop()
      if isinstance(self.history["move"][-1], list) and len(self.history["move"][-1]) == 4:
        k = self.history["move"][-1][3]
        self.Ko = [k[0], k[1]]
      if move[0] == "Pass":
        self.__pass -=1
        assert self.__pass >= 0, "_Game__pass<0"
      else:
        if self.history["move"][-1] and self.history["move"][-1][0] == "Pass":
          self.__pass = 1
    self.regroup_all()
    self.show_board = self.board
    self.state = "Proceeding"
    self.__legal_moves = None
    self.moves()

  def forward(self, n):
    if n and self.step < len(self.history["board"])-n:
      self.step += n
      self.show_board = self.history["board"][self.step]
      return True
    self.step = len(self.history["board"])-1
    self.show_board = self.history["board"][-1]
    return False

  def backward(self, n):
    if n and self.step>n-1:
      self.step -= n
      self.show_board = self.history["board"][self.step]
      return True
    self.step = 0
    self.show_board = self.history["board"][0]
    return False

  def moves(self):
    if self.__legal_moves != None:
      return self.__legal_moves

    m = []
    for x in range(self.size):
      for y in range(self.size):
        if self.board[x][y] == 0:
          eye  = self.check_eye([x,y])
          if self.check_Qi([x,y]) > 0 and not eye:
            m.append([x,y])
    if self.Ko:
      if self.Ko in m:
        m.remove(self.Ko)
      self.Ko = None
    m.append("Pass")
    self.__legal_moves = m
    return self.__legal_moves


  def check_Atari(self):
    return

  def check_Ko(self, move):
    x, y = move
    if len(self.group_[x][y]["qi"]) == 1 and len(self.group_[x][y]["stone"]) == 1:
      self.Ko = list(list(self.group_[x][y]["qi"])[0])

  def check_Qi(self, move):
    x, y = move
    qi = 0
    grouping = []
    if x>0:
      if self.board[x-1][y] == -self.turn:
        g = self.group_[x-1][y]
        if len(g["qi"]) == 1: qi += 1
      elif self.board[x-1][y] == self.turn:
        g = self.group_[x-1][y]
        gg = False
        for k in grouping:
          if g == k:
            gg = True
            break
        if not gg:
          qi += len(self.group_[x-1][y]["qi"])-1
          grouping.append(g)
      elif  self.board[x-1][y] == 0:
        qi += 1

    if x<len(self.board)-1:
      if self.board[x+1][y] == -self.turn:
        g = self.group_[x+1][y]
        if len(g["qi"]) == 1: qi += 1
      elif self.board[x+1][y] == self.turn:
        g = self.group_[x+1][y]
        gg = False
        for k in grouping:
          if g == k:
            gg = True
            break
        if not gg:
          qi += len(self.group_[x+1][y]["qi"])-1
          grouping.append(g)
      elif  self.board[x+1][y] == 0:
        qi += 1

    if y>0:
      if self.board[x][y-1] == -self.turn:
        g = self.group_[x][y-1]
        if len(g["qi"]) == 1: qi += 1
      elif self.board[x][y-1] == self.turn:
        g = self.group_[x][y-1]
        gg = False
        for k in grouping:
          if g == k:
            gg = True
            break
        if not gg:
          qi += len(self.group_[x][y-1]["qi"])-1
          grouping.append(g)
      elif  self.board[x][y-1] == 0:
        qi += 1

    if y<len(self.board[0])-1:
      if self.board[x][y+1] == -self.turn:
        g = self.group_[x][y+1]
        if len(g["qi"]) == 1: qi += 1
      elif self.board[x][y+1] == self.turn:
        g = self.group_[x][y+1]
        gg = False
        for k in grouping:
          if g == k:
            gg = True
            break
        if not gg:
          qi += len(self.group_[x][y+1]["qi"])-1
          grouping.append(g)
      elif  self.board[x][y+1] == 0:
        qi += 1

    return qi



  def check_Kill(self, move):
    def remove(group):
      for x,y in group["stone"]:
        if x>0 and self.board[x-1][y] == self.turn:
          if self.group_[x-1][y]:
            self.group_[x-1][y]["qi"].add((x,y))
        if x<len(self.board)-1 and self.board[x+1][y] == self.turn:
          if self.group_[x+1][y]:
            self.group_[x+1][y]["qi"].add((x,y))
        if y>0 and self.board[x][y-1] == self.turn:
          if self.group_[x][y-1]:
            self.group_[x][y-1]["qi"].add((x,y))
        if y<len(self.board[0])-1 and self.board[x][y+1] == self.turn:
          if self.group_[x][y+1]:
            self.group_[x][y+1]["qi"].add((x,y))

        self.board[x][y] = 0
        self.group_[x][y] = None

    count = 0
    x, y = move
    if x>0 and self.board[x-1][y] == -self.turn:
      g = self.group_[x-1][y]
      if len(g["qi"]) == 0:
        count += len(g["stone"])
        remove(g)

    if x<len(self.board)-1 and self.board[x+1][y] == -self.turn:
      g = self.group_[x+1][y]
      if len(g["qi"]) == 0:
        count += len(g["stone"])
        remove(g)
    if y>0 and self.board[x][y-1] == -self.turn:
      g = self.group_[x][y-1]
      if len(g["qi"]) == 0:
        count += len(g["stone"])
        remove(g)
    if y<len(self.board[0])-1 and self.board[x][y+1] == -self.turn:
      g = self.group_[x][y+1]
      if len(g["qi"]) == 0:
        count += len(g["stone"])
        remove(g)

    return count

  def check_Win(self):
    if self.history["move"][-1][0]=="Draw":
      return 0
      
    b, w = self.check_Land()
    if b>w:
      return 1
    elif w>b:
      return -1
    else:
      return 0 #draw

  # Todos:
  # consider seki(mutual life)
  # definition of eye?
  # vacancies are equally divided by their neighbors.
  def check_Land(self):
    black = 0
    black_eye = set()
    white = 0
    white_eye = set()
    grouping = []
    for x in range(self.size):
      for y in range(self.size):
        gxy = self.group_[x][y]
        if gxy:
          if len(gxy["eye"])>=2 and not gxy in grouping:
            if gxy["player"] == 1:
              black += len(gxy["stone"])
              black_eye |= gxy["eye"]
            elif gxy["player"] == -1:
              white += len(gxy["stone"])
              white_eye |= gxy["eye"]
            grouping.append(gxy)
    black += len(black_eye)
    white += len(white_eye)
    return black, white

  def check_LandX(self):
    black = 0
    white = 0
    for x in range(self.size):
      for y in range(self.size):
        Bxy = self.board[x][y]
        if Bxy == 1:
          black += 1
        elif Bxy == -1:
          white += 1
        elif Bxy == 0:
          b0 = 0
          w0 = 0
          if x > 0:
            Bxy = self.board[x-1][y]
            if Bxy == 1:
              b0 += 1
            elif Bxy == -1:
              w0 += 1
          if x < self.size-1:
            Bxy = self.board[x+1][y]
            if Bxy == 1:
              b0 += 1
            elif Bxy == -1:
              w0 += 1            
          if y > 0:
            Bxy = self.board[x][y-1]
            if Bxy == 1:
              b0 += 1
            elif Bxy == -1:
              w0 += 1    
          if y < self.size-1:
            Bxy = self.board[x][y+1]
            if Bxy == 1:
              b0 += 1
            elif Bxy == -1:
              w0 += 1 

          if b0 > 0 and w0 == 0:
            black += 1
          if b0 == 0 and w0 > 0:
            white += 1                 
    return black, white

  def neighbors(self, x,y):
    assert x>=0 and x<self.size, "x is out of range!"
    assert y>=0 and y<self.size, "y is out of range!"
    ne = []
    if x>0:
      ne.append([x-1, y])

    if x<self.size-1:
      ne.append([x+1,y])

    if y>0:
      ne.append([x,y-1])

    if y<self.size-1:
      ne.append([x,y+1])

    return ne

  def check_eye(self, move):
  # a vacancy surrounded by 1 group only is considered a complete eye.
  # a group with 2 complete eyes or more is a living group, which is no way to be killed.
  # a vacancy surrounded by more than 1 group is considered an incomplete eye, denoted "ey".
  # groups have chance to be merged and become a living group, if the 
  # following conditions are met:
  # 1. each group in those groups share 2 or more ey's with each other
  # 2. those common ey's are surrounded by groups in those groups only.
  # 3. if the above conditions are met, those common ey's can be upgraded to complete eyes,
  #   and those groups can be merged as one group which is considered a living group.

    def merge(grouping):
      new_eye = set()
      stranger = []
      common = [set() for i in range(len(grouping))]
      for i in range(len(grouping)):
        ey = grouping[i]["ey"]
        for j in range(i+1, len(grouping)):
          ey_ = grouping[j]["ey"]
          ey_ey = ey & ey_
          for eyX,eyY in ey_ey:
            for nx,ny in self.neighbors(eyX,eyY):
              g = self.group_[nx][ny]
              if g and g not in grouping:
                if g["player"] == grouping[0]["player"]:
                  if g not in stranger:
                    stranger.append(self.group_[nx][ny])
            if not stranger:   
              common[i].add((eyX,eyY))
              common[j].add((eyX,eyY))

      #for i in range(len(grouping)):
      #  print(i, grouping[i])
      #  print("    ",common[i])
      for i in range(len(grouping)):
        if len(common[i])<2: return False, stranger
      for i in range(len(grouping)):
        for co in common[i]:
          new_eye.add(co)
      for eyX,eyY in new_eye:
        for nx,ny in self.neighbors(eyX,eyY):
          g = self.group_[nx][ny]
          if g: g["eye"].add((eyX,eyY))
      return True, stranger


    x, y = move
    grouping = []
    turn = 0
    for nx,ny in self.neighbors(x,y):
      g = self.group_[nx][ny]
      if not g: return False
      if g not in grouping: grouping.append(g)

    turn = grouping[0]["player"]
    for g in grouping:
      if g["player"] !=turn: return False
    

    for g in grouping:
      if (x,y) not in g["ey"]: g["ey"].add((x,y))

    if len(grouping) == 1:  
      g["eye"].add((x,y))
    else:
      isEye = False
      stranger = True
      n=5
      while stranger:
        #print("n: ",n)
        merged, stranger= merge(grouping)
        grouping += stranger
        if merged: isEye=True
        n -= 1
        #assert n>0, "merge loop..."
      if not isEye: return False
          
    if turn != self.turn: return False
    return True

  def check_eyeX(self, move):
    x, y = move
    g = False
    if x>0:
      if self.board[x-1][y] != self.turn: return False
      if g and g!=self.group_[x-1][y]: return False
      g = self.group_[x-1][y]
    if x<len(self.board)-1:
      if self.board[x+1][y] != self.turn: return False
      if g and g!=self.group_[x+1][y]: return False
      g = self.group_[x+1][y]
    if y>0:
      if self.board[x][y-1] != self.turn: return False
      if g and g!=self.group_[x][y-1]: return False
      g = self.group_[x][y-1]
    if y<len(self.board[0])-1:
      if self.board[x][y+1] != self.turn: return False
      if g and g!=self.group_[x][y+1]: return False
      g = self.group_[x][y+1]

    if not move in g["eye"]:
      g["eye"].append(move)
    return True


  def test_move(self, color, move):
    if color != self.turn:
      print(color, ": ", move, " not my turn!")
      return

    print(self.turn,": ", end="")
    self.make_move(move)
    print(move, self.__pass, self.Ko)



  def test(self):
    print("turn: ", "move: ","pass: ","Ko: ")
    self.test_move(1, [0,0])
    self.test_move(-1, [0,1])
    self.test_move(1, [1,1])
    self.test_move(-1, [1,2])
    self.test_move(1, [0,2])
    self.test_move(-1, [0,3])
    self.test_move(1, [2,0])
    self.test_move(-1, [0,1])
    self.test_move(1, "Pass")
    self.test_move(-1, [4,4])
    self.test_move(1, [2,1])
    self.test_move(-1, [1,3])
    self.test_move(1, [0,2])
    self.test_move(-1, "Pass")
    self.test_move(1, "Pass")
    self.test_move(-1, [6,6])

  def test_eye(self):
    test_eye_moves = [ [1, 1, 0],[4, 3, 0],[0, 0, 0],[3, 4, 0],\
              [0, 2, 0],[2, 3, 0],[2, 0, 0],[3, 2, 0],\
              [1, 2, 0],[2, 4, 0],[2, 1, 0],[4, 2, 0],\
              [3, 1, 0],[2, 2, 0],[4, 1, 0],[1, 3, 0],\
              [4, 0, 0]]


    turn = 1
    print("turn: ", "move: ","pass: ","Ko: ")
    for m in test_eye_moves:
      if m[0]=="Pass":
        move ="Pass"
      else:
        move = [m[0],m[1]]
      self.test_move(turn, move)
      turn = -turn

    for m in test_eye_moves:
      if m[0]!="Pass":
        x,y = m[0],m[1]
        try:
          print(m,"-> eye: ",self.group_[x][y]["eye"],"ey: ",self.group_[x][y]["ey"])
        except:
          print("Error: self.group_[",x,"][",y,"] =", self.group_[x][y])

  def test_eye_2(self):
    test_eye_moves = [ [1, 1, 0],[4, 3, 0],[0, 0, 0],[3, 4, 0],\
              [0, 2, 0],[2, 3, 0],[2, 0, 0],[3, 2, 0],\
              [1, 2, 0],[2, 4, 0],[2, 1, 0],[4, 2, 0],\
              [3, 1, 0],[2, 2, 0],[4, 1, 0],[1, 3, 0],\
              [4, 0, 0]]
    test_eye_moves_2 = [[1, 1, 0],[4, 4, 0],[0, 0, 0],[3, 3, 0],[2, 0, 0],[2, 3, 0],\
              [0, 2, 0],[2, 4, 0],[1, 2, 0],[4, 2, 0],[2, 1, 0],[3, 1, 0]]

    turn = 1
    print("turn: ", "move: ","pass: ","Ko: ")
    for m in test_eye_moves_2:
      if m[0]=="Pass":
        move ="Pass"
      else:
        move = [m[0],m[1]]
      self.test_move(turn, move)
      turn = -turn

    for m in test_eye_moves_2:
      if m[0]!="Pass":
        x,y = m[0],m[1]
        try:
          print(m,"-> eye: ",self.group_[x][y]["eye"],"ey: ",self.group_[x][y]["ey"])
        except:
          print("Error: self.group_[",x,"][",y,"] =", self.group_[x][y])

  def test_eye_3(self):
    #test_eye_moves_3 =[[0, 1, 0],[3, 1, 0],[1, 0, 0],[4, 2, 0],\
    #          [2, 1, 0],[3, 3, 0],[3, 0, 0],[2, 2, 0],\
    #          [0, 0, 0],[2, 4, 0],[1, 2, 0],[1, 3, 0],\
    #          [0, 2, 0],[1, 4, 0],[2, 0, 0],[0, 4, 0],\
    #          [4, 0, 0],[4, 1, 0],[0, 3, 0],[4, 4, 0],\
    #          ['Pass', 0],[4, 3, 0],['Pass', 0]]

    test_eye_moves_3 = [[0, 1, 0],[4, 4, 0],[1, 0, 0],[3, 4, 0],[2, 1, 0],\
              [4, 3, 0],[1, 2, 0],[3, 3, 0],[3, 0, 0],[2, 4, 0],\
              [0, 3, 0],[4, 2, 0],[1, 3, 0],[1, 4, 0],[3, 1, 0],[4, 1, 0]]
    turn = 1
    print("turn: ", "move: ","pass: ","Ko: ")
    for m in test_eye_moves_3:
      if m[0]=="Pass":
        move ="Pass"
      else:
        move = [m[0],m[1]]
      self.test_move(turn, move)
      turn = -turn

    for m in test_eye_moves_3:
      if m[0]!="Pass":
        x,y = m[0],m[1]
        try:
          print(m,"-> eye: ",self.group_[x][y]["eye"],"ey: ",self.group_[x][y]["ey"])
        except:
          print("Error: self.group_[",x,"][",y,"] =", self.group_[x][y])




#class Node():
#  def __init__(self, position, value):
#    self.p = position
#    self.v = value
#    self.children = []




In [4]:
g=Game(5)
g.test()

turn:  move:  pass:  Ko: 
1 : [0, 0] 0 None
-1 : [0, 1] 0 None
1 : [1, 1] 0 None
-1 : [1, 2] 0 None
1 : [0, 2] 0 None
-1 : [0, 3] 0 None
1 : [2, 0] 0 None
-1 : [0, 1] 0 [0, 2]
1 : Pass 1 None
-1 : [4, 4] 0 None
1 : [2, 1] 0 None
-1 : [1, 3] 0 None
1 : [0, 2] 0 [0, 1]
-1 : Pass 1 None
1 : Pass 2 None
-1 : [6, 6] 2 None


In [5]:
g=Game(5)
g.test_eye_2()

turn:  move:  pass:  Ko: 
1 : [1, 1] 0 None
-1 : [4, 4] 0 None
1 : [0, 0] 0 None
-1 : [3, 3] 0 None
1 : [2, 0] 0 None
-1 : [2, 3] 0 None
1 : [0, 2] 0 None
-1 : [2, 4] 0 None
1 : [1, 2] 0 None
-1 : [4, 2] 0 None
1 : [2, 1] 0 None
-1 : [3, 1] 0 None
[1, 1, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[4, 4, 0] -> eye:  set() ey:  {(3, 4), (4, 3)}
[0, 0, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[3, 3, 0] -> eye:  set() ey:  {(3, 4), (4, 3)}
[2, 0, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[2, 3, 0] -> eye:  set() ey:  {(3, 4), (4, 3)}
[0, 2, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[2, 4, 0] -> eye:  set() ey:  {(3, 4), (4, 3)}
[1, 2, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[4, 2, 0] -> eye:  set() ey:  {(4, 3)}
[2, 1, 0] -> eye:  {(0, 1), (1, 0)} ey:  {(0, 1), (1, 0)}
[3, 1, 0] -> eye:  set() ey:  set()


In [6]:
g=Game(5)
g.test_eye_3()

turn:  move:  pass:  Ko: 
1 : [0, 1] 0 None
-1 : [4, 4] 0 None
1 : [1, 0] 0 None
-1 : [3, 4] 0 None
1 : [2, 1] 0 None
-1 : [4, 3] 0 None
1 : [1, 2] 0 None
-1 : [3, 3] 0 None
1 : [3, 0] 0 None
-1 : [2, 4] 0 None
1 : [0, 3] 0 None
-1 : [4, 2] 0 None
1 : [1, 3] 0 None
-1 : [1, 4] 0 None
1 : [3, 1] 0 None
-1 : [4, 1] 0 None
[0, 1, 0] -> eye:  {(0, 0), (0, 2), (1, 1)} ey:  {(0, 2), (0, 0), (1, 1)}
[4, 4, 0] -> eye:  set() ey:  set()
[1, 0, 0] -> eye:  {(2, 0), (0, 0), (1, 1)} ey:  {(2, 0), (0, 0), (1, 1)}
[3, 4, 0] -> eye:  set() ey:  set()
[2, 1, 0] -> eye:  {(2, 0), (1, 1)} ey:  {(2, 0), (1, 1)}
[4, 3, 0] -> eye:  set() ey:  set()
[1, 2, 0] -> eye:  {(1, 1), (0, 2)} ey:  {(0, 2), (1, 1)}
[3, 3, 0] -> eye:  set() ey:  set()
[3, 0, 0] -> eye:  {(2, 0), (1, 1)} ey:  {(2, 0), (1, 1)}
[2, 4, 0] -> eye:  set() ey:  set()
[0, 3, 0] -> eye:  {(1, 1), (0, 2)} ey:  {(0, 2), (1, 1)}
[4, 2, 0] -> eye:  set() ey:  set()
[1, 3, 0] -> eye:  {(1, 1), (0, 2)} ey:  {(0, 2), (1, 1)}
[1, 4, 0] -> eye:  set()

# Player

In [7]:
class Player():
  def __init__(self, color, game):
    self.color = color
    self.game = game
    self.mc = MCTS(self.game)
    self.dn = DNN(self.game)

  def random(self):
    if self.color != self.game.turn:
      print(self.color, ": not my turn!")
      return

    self.game.make_move(random.choice(self.game.moves()))
  


  def mcts(self):
    move, score = self.mc.act()
    self.game.make_move(move)
    m = self.game.history["move"][self.game.step]
    if isinstance(m[0],str):
      m[1] = score
    else:
      m[2] = score
    return True

  def dnn(self):
    move, score = self.dn.act()
    self.game.make_move(move)
    m = self.game.history["move"][self.game.step]
    if m[0] == "Pass":
      m[1] = score
    else:
      m[2] = score
    return True


# MCTS

In [8]:
print(sys.getrecursionlimit())
#sys.setrecursionlimit(1000)

1000


In [9]:
%%script false --no-raise-error

# the node class stores a list of available moves
# and the associated play counts and scores for
# each move. 

class Node():
  # by default, nodes are initialised as leaves and as non-terminal states
  def __init__(self):
    self.leaf = True
    self.terminal = False #unnecessary?

  # A node is expanded using a list of moves.
  # The node is terminal if there are no moves (game drawn).
  # Otherwise, the scores for the moves are initialised to zero
  # and the counters to a small number to avoid division by zero.
  # One child is created for each move.
  def expand(self, moves):
    m = len(moves)
    #if m == 0:
    #  self.terminal = True
    #at least there is a move named "Pass"
    if m:
      # S stores the cumulative reward for each of the m moves 
      self.S = numpy.zeros(m)

      # T stores the number of plays for each of the m moves
      self.T = numpy.full(m, 0.001)

      # moves stores the list of moves available in this node
      self.moves = deepcopy(moves)

      self.children = [Node() for a in range(m)]
      self.leaf = False

  # when the move associated with idx is played
  # and a score obtained, this function should
  # be used to update the counts (T) and scores (S)
  # associated with the idx
  def update(self, idx, score):
    self.S[idx]+=score
    self.T[idx]+=1

  # THIS FUNCTION NEEDS IMPLEMENTING
  # This function decides which node to search using a bandit algorithm
  # Notes:
  # (1) self.S stores the cumulative returns of each available move at this node
  # (2) self.T stores the number of times each available move has been explored
  # (3) numpy.argmax(v) returns the coordinate the maximises vector v.
  # (4) numpy is quite permissive about operations on vectors. When v and w have the same size, then v/w divides
  # coordinatewise. Similarly, numpy.sqrt(v) is a coordinate-wise operation.
  # (5) You might like experimenting with this function
  def choose(self):
    idx = numpy.argmax(self.S / self.T + numpy.sqrt(2.0 / self.T * numpy.log(1 + self.T.sum())))
    return idx


class MCTS():
  def __init__(self, game, iterations = 1000):
    self.game = game
    self.turn = game.turn
    self.iterations = iterations
    self.max_game_step = 0
    self.max_mcts = 0
    self.max_rollout = 0
    self.n_mcts = 0
    self.n_rollout = 0

  def act(self):
    #assert self.game.size != 3, "infinite loop is found in a 3x3 game"
    assert self.game.state != "Over", "Game is over!"
    self.current_step = self.game.step
    result = self.search() 
    assert self.current_step == self.game.step, "game is at not current step {current_step}"
    return result

  def feed(self, move):
    self.game.make_move(move)

  def search(self):
    #print("MCTS searching ", self.turn,": ", self.game.step)
    # create the root note
    node = Node() 
    self.node = node
    # repeat a number of iterations of MCTS
    for t in range(self.iterations):
      self.mcts(node)

    # find the index of the most played move at the root node and associated move
    #idx = numpy.argmax(node.T)
  
    idx = numpy.argmax(node.S/node.T) #James
    move = node.moves[idx]

    # print the average return of this move and return the move
    score = node.S[idx] / node.T[idx]
    return move, score


  def mcts(self, node):
    self.n_mcts += 1
    if self.n_mcts > self.max_mcts: self.max_mcts = self.n_mcts


    # if the node is a leaf, then expand it
    if node.leaf:
      node.expand(self.game.moves())
      rollout = True
    else:
      rollout = False

    if node.terminal:
      return 0

    if self.n_mcts+self.n_rollout > 900:
      self.n_mcts -= 1
      return 0

    # choose a move 
    idx = node.choose()
    move = node.moves[idx]

    winner = self.game.make_move(move)
    if self.game.state == "Over":
      if self.game.step > self.max_game_step: self.max_game_step = self.game.step
      if self.game.turn == -winner:
        val = 1
      elif self.game.turn == winner:
        val = -1
      elif winner == 0:
        val = 0 #draw
      else: assert False
    elif rollout: 
      val = -self.rollout()
    else:
      val = -self.mcts(node.children[idx])

    self.game.unmake_move()
    # update the value of the associated move
    node.update(idx, val)
    self.n_mcts -= 1
    return val

  # THIS FUNCTION NEEDS IMPLEMENTING
  # returns the result of a Monte-Carlo rollout from the current game state
  # until the game ends. After the function has returned, the game-state should
  # not be different than it was at the start.
  # The value should be from the perspective of the player currently to move:
  # (1) If the player wins, it should be 1
  # (2) If the player loses, it should be -1
  # (3) If the game was drawn, it should be 0
  # Note:
  # self.game.moves() returns the moves available in the current position
  # self.game.make_move(move) makes move on self.game and returns True if that move won the game
  # self.game.unmake_move() unmakes the last move in self.game
  # Be care that rollout should return the score from the perspective of the player currently to move
  def rollout(self):
    self.n_rollout += 1
    if self.n_rollout > self.max_rollout: self.max_rollout = self.n_rollout
    if self.n_mcts+self.n_rollout > 900:
      self.n_rollout -= 1
      return 0

    moves = self.game.moves()
    move = random.choice(moves)
    winner = self.game.make_move(move)
    if self.game.state == "Over":
      if self.game.step > self.max_game_step: self.max_game_step = self.game.step
      if self.game.turn == -winner:
        val = 1
      elif self.game.turn == winner:
        val= -1
      elif winner == 0:
        val = 0 #draw
      else: assert False
    else: 
      val = -self.rollout()       # otherwise take the negative of a rollout continuing from here

    self.game.unmake_move()
    #print("r ",self.game.step,": ",val)
    self.n_rollout -= 1
    return val

  # You can call this function to test your rollout function
  # Running it should give results of about 0.1 +- 0.05 and -0.03 +- 0.01
  def test_rollout(self):
    assert(len(self.game.history["move"]) == 1)
    n = 1000
    res = 0.0
    for t in range(n):
      res+=self.rollout()
      assert(len(self.game.history["move"]) == 1)
    print("average result from start with random play: " + str(res / n)) 
    self.game.make_move([0,0])
    self.turn = self.game.turn
    res = 0.0
    for t in range(n):
      res+=self.rollout()
      assert(len(self.game.history["move"]) == 2)
    print("average result after first player moves 0: " + str(res / n))


In [10]:
# the node class stores a list of available moves
# and the associated play counts and scores for
# each move. 

class Node():
  # by default, nodes are initialised as leaves and as non-terminal states
  def __init__(self):
    self.leaf = True
    self.terminal = False #unnecessary?

    self.children = []
    self.board = []
    self.moves = []
    self.S = numpy.array([])
    self.T = numpy.array([])

  # A node is expanded using a list of moves.
  # The node is terminal if there are no moves (game drawn).
  # Otherwise, the scores for the moves are initialised to zero
  # and the counters to a small number to avoid division by zero.
  # One child is created for each move.
  def expand(self, moves):
    m = len(moves)
    #if m == 0:
    #  self.terminal = True
    #at least there is a move named "Pass"
    if m:
      # S stores the cumulative reward for each of the m moves 
      self.S = numpy.zeros(m)

      # T stores the number of plays for each of the m moves
      self.T = numpy.full(m, 0.001)

      # moves stores the list of moves available in this node
      self.moves = deepcopy(moves)

      self.children = [Node() for a in range(m)]
      self.leaf = False

  # when the move associated with idx is played
  # and a score obtained, this function should
  # be used to update the counts (T) and scores (S)
  # associated with the idx
  def update(self, idx, score):
    self.S[idx]+=score
    self.T[idx]+=1

  # THIS FUNCTION NEEDS IMPLEMENTING
  # This function decides which node to search using a bandit algorithm
  # Notes:
  # (1) self.S stores the cumulative returns of each available move at this node
  # (2) self.T stores the number of times each available move has been explored
  # (3) numpy.argmax(v) returns the coordinate the maximises vector v.
  # (4) numpy is quite permissive about operations on vectors. When v and w have the same size, then v/w divides
  # coordinatewise. Similarly, numpy.sqrt(v) is a coordinate-wise operation.
  # (5) You might like experimenting with this function
  def choose(self):
    idx = numpy.argmax(self.S / self.T + numpy.sqrt(2.0 / self.T * numpy.log(1 + self.T.sum())))
    return idx


class MCTS():
  def __init__(self, game, iterations = 1000):
    self.game = game
    self.turn = game.turn
    self.iterations = iterations
    self.max_game_step = 0
    self.max_mcts = 0
    self.max_rollout = 0
    self.n_mcts = 0
    self.n_rollout = 0
    self.node = Node()

  def act(self):
    #assert self.game.size != 3, "infinite loop is found in a 3x3 game"
    assert self.game.state != "Over", "Game is over!"
    self.current_step = self.game.step
    result = self.search() 
    assert self.current_step == self.game.step, "game is at not current step {current_step}"
    return result

  def feed(self, move):
    self.game.make_move(move)

  def search(self):
    #print("MCTS searching ", self.turn,": ", self.game.step)
    # create the root note
    node = Node() 
    self.node = node
    # repeat a number of iterations of MCTS
    for t in range(self.iterations):
      self.mcts(node)

    # find the index of the most played move at the root node and associated move
    #idx = numpy.argmax(node.T)
  
    idx = numpy.argmax(node.S/node.T) #James
    move = node.moves[idx]

    # print the average return of this move and return the move
    score = node.S[idx] / node.T[idx]
    return move, score


  def mcts(self, node):
    self.n_mcts += 1
    if self.n_mcts > self.max_mcts: self.max_mcts = self.n_mcts

    node.board = deepcopy(self.game.board) #James

    # if the node is a leaf, then expand it
    if node.leaf:
      node.expand(self.game.moves())
      rollout = True
    else:
      rollout = False

    if node.terminal:
      return 0

    if self.n_mcts+self.n_rollout > 900:
      self.n_mcts -= 1
      return 0

    # choose a move 
    idx = node.choose()
    move = node.moves[idx]

    winner = self.game.make_move(move)
    if self.game.state == "Over":
      if self.game.step > self.max_game_step: self.max_game_step = self.game.step
      if self.game.turn == -winner:
        val = 1
      elif self.game.turn == winner:
        val = -1
      elif winner == 0:
        val = 0 #draw
      else: assert False
    elif rollout: 
      val = -self.rollout()
    else:
      val = -self.mcts(node.children[idx])

    self.game.unmake_move()
    # update the value of the associated move
    node.update(idx, val)
    self.n_mcts -= 1
    return val

  # THIS FUNCTION NEEDS IMPLEMENTING
  # returns the result of a Monte-Carlo rollout from the current game state
  # until the game ends. After the function has returned, the game-state should
  # not be different than it was at the start.
  # The value should be from the perspective of the player currently to move:
  # (1) If the player wins, it should be 1
  # (2) If the player loses, it should be -1
  # (3) If the game was drawn, it should be 0
  # Note:
  # self.game.moves() returns the moves available in the current position
  # self.game.make_move(move) makes move on self.game and returns True if that move won the game
  # self.game.unmake_move() unmakes the last move in self.game
  # Be care that rollout should return the score from the perspective of the player currently to move
  def rollout(self):
    self.n_rollout += 1
    if self.n_rollout > self.max_rollout: self.max_rollout = self.n_rollout
    if self.n_mcts+self.n_rollout > 900:
      self.n_rollout -= 1
      return 0

    moves = self.game.moves()
    move = random.choice(moves)
    winner = self.game.make_move(move)
    if self.game.state == "Over":
      if self.game.step > self.max_game_step: self.max_game_step = self.game.step
      if self.game.turn == -winner:
        val = 1
      elif self.game.turn == winner:
        val= -1
      elif winner == 0:
        val = 0 #draw
      else: assert False
    else: 
      val = -self.rollout()       # otherwise take the negative of a rollout continuing from here

    self.game.unmake_move()
    #print("r ",self.game.step,": ",val)
    self.n_rollout -= 1
    return val

  # You can call this function to test your rollout function
  # Running it should give results of about 0.1 +- 0.05 and -0.03 +- 0.01
  def test_rollout(self):
    assert(len(self.game.history["move"]) == 1)
    n = 1000
    res = 0.0
    for t in range(n):
      res+=self.rollout()
      assert(len(self.game.history["move"]) == 1)
    print("average result from start with random play: " + str(res / n)) 
    self.game.make_move([0,0])
    self.turn = self.game.turn
    res = 0.0
    for t in range(n):
      res+=self.rollout()
      assert(len(self.game.history["move"]) == 2)
    print("average result after first player moves 0: " + str(res / n))


# Reinforcement Learning

In [11]:
%%script false --no-raise-error
class DNN():
  def __init__(self, game):
    self.game = game
    #self.model = keras.models.load_model("/content/drive/MyDrive/Colab/model_0923")

    # Value Network
    if False:
      self.value = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(5,5,1)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)])
      self.value.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy'])
    else:
      self.value = keras.models.load_model("/content/drive/MyDrive/Colab/value_network_0923")


    # Policy Network
    if False:
      self.policy = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(5,5,1)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(25)])
      self.policy.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy'])
    else:
      self.policy = keras.models.load_model("/content/drive/MyDrive/Colab/policy_network_0923")


  def act(self):
    self.moves = deepcopy(self.game.moves())     
    self.actions = self.policy.predict(numpy.array([self.game.board]))

    self.moves.remove("Pass")
    if self.moves == []: return "Pass", 0
    self.values = numpy.zeros(len(self.moves))
    for i in range(len(self.moves)):
      x,y = self.moves[i]
      self.values[i] = self.actions[0][x*5+y]
      
    if self.game.turn == 1:
      idx = numpy.argmax(self.values)
      return self.moves[idx], self.values[idx]
    if self.game.turn == -1:
      idx = numpy.argmin(self.values)
      return self.moves[idx], self.values[idx]
    assert False, "invalid turn"

  def train_value(self):
    with open("/content/drive/MyDrive/Colab/value_board_train_0924.json") as fp:
      value_board = json.load(fp)
    v= []
    b= []
    for vb in value_board:
      v.append(vb[0])
      b.append(vb[1])
    train_values = numpy.array(v)
    print(train_values.shape)
    train_boards = numpy.array(b)
    print(train_boards.shape)

    with open("/content/drive/MyDrive/Colab/value_board_tests_0923.json") as fp:
      tests = json.load(fp)
    v = []
    b = []
    for vb in tests:
      v.append(vb[0])
      b.append(vb[1])
    test_values = numpy.array(v)
    test_boards = numpy.array(b)
    print(test_boards.shape)
    print(test_values.shape)

    history = self.value.fit(train_boards, train_values, epochs=100, 
                    validation_data=(test_boards, test_values))
    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.0, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = self.value.evaluate(test_boards,  test_values, verbose=2)
    
  def train_policy(self):
    self.n_rollout=0
    def rollout():
      self.n_rollout += 1
      if self.n_rollout > 100:
        self.n_rollout -= 1
        return 0

      moves = self.game.moves()
      actions = [[0 for i in range(self.game.size)] for j in range(self.game.size)]
      for m in moves:
        if isinstance(m,list):
          x,y = m
          bb = deepcopy(self.game.board)
          bb[x][y]=self.game.turn
          actions[x][y] = float(self.value.predict(numpy.array([bb]))[0][0])
      self.action_board.append([actions, deepcopy(self.game.board)])

      move = random.choice(moves)
      winner = self.game.make_move(move)
      if self.game.state == "Over":
        if self.game.turn == -winner:
          val = 1
        elif self.game.turn == winner:
          val= -1
        elif winner == 0:
          val = 0 #draw
        else: assert False
      else: 
        val = -rollout()
      self.game.unmake_move()
      self.n_rollout -= 1
      return val   

    self.action_board = []
    for i in range(1):
      rollout()

    with open("/content/drive/MyDrive/Colab/action_board_train_0925.json", "w") as fp:
      json.dump(self.action_board, fp)

    a = []
    b = []
    for ab in self.action_board:
      a.append(numpy.array(ab[0]).flatten())
      b.append(ab[1])
    train_actions = numpy.array(a)
    train_boards = numpy.array(b)

    history = self.policy.fit(train_boards, train_actions, epochs=10,)
    
     
  def save(self):
    self.value.save("/content/drive/MyDrive/Colab/value_network_0923")
    self.policy.save("/content/drive/MyDrive/Colab/policy_network_0923")


In [12]:
class DNN():
  def __init__(self, game):
    self.game = game

    # PV Network
    if False:
      self.pv = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(5,5,1)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(26)]) #1~25:moves on game board, 26:"Pass"
      self.pv.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy'])
    else:
      self.pv = keras.models.load_model("/content/drive/MyDrive/Colab/pv_network_0927")


  def act(self):
    self.moves = deepcopy(self.game.moves())     
    self.actions = self.pv.predict(numpy.array([self.game.board]))[0]
    self.values = [0 for i in range(len(self.moves))]

    self.p = numpy.zeros(len(self.moves))
    for i in range(len(self.moves)):
      if self.moves[i] != "Pass":
        x,y = self.moves[i]
        self.values[i] = self.actions[x*5+y]
      else:
        self.values[i] = self.actions[25]
      
    if self.game.turn == 1:
      idx = numpy.argmax(self.values)
      return self.moves[idx], self.values[idx]
    if self.game.turn == -1:
      idx = numpy.argmin(self.values)
      return self.moves[idx], self.values[idx]
    assert False, "invalid turn"


  def mcts(self):
    def TS2ar(node):
      a0 = [0 for i in range(26)]
      r0 = [0 for i in range(26)]
      for i in range(len(node.moves)):
        if node.moves[i] != "Pass":
          x,y = node.moves[i]
          a0[x*5+y] = node.T[i]
          r0[x*5+y] = (node.S/node.T)[i]
      return a0, r0

    def traverse(node):
      for child in node.children:
        if child.board:
          a.append(TS2ar(child)[0])
          b.append(child.board)
          r.append(TS2ar(child)[1])
          traverse(child)
      return

    self.mcts = MCTS(self.game)

    b = []
    a = []
    r = []
    while self.game.state != "Over":
      print("MCTS: ", self.game.step)
      move, score = self.mcts.act()
      root = self.mcts.node

      b.append(root.board)
      a.append(TS2ar(root)[0])
      r.append(TS2ar(root)[1])

      #traverse(root)

      self.game.make_move(move)

    with open("/content/drive/MyDrive/Colab/pv_train_0927", "w") as fp:
      json.dump({"board":b,"actions":a,"return":r}, fp)

  def train_pv(self):
    with open("/content/drive/MyDrive/Colab/pv_train_0927", "r") as fp:
      board_action = json.load(fp)

    b = board_action["board"]
    self.b = b
    a = board_action["actions"]
    r = board_action["return"]

    if False:
      numpy.set_printoptions(precision=3)
      print("Total steps:", len(b))
      for i in range(len(b)):
        print("Step: ",i)
        print(len(a[i]),len(r[i]))
        for j in range(5):
          print(b[i][j], numpy.array(a[i][j*5:j*5+5]), numpy.array(r[i][j*5:j*5+5]))
        print("Pass", numpy.array(a[i][-1]), numpy.array(r[i][-1]))
    train_actions = numpy.array(a)
    train_return = numpy.array(r)
    train_boards = numpy.array(b)

    history = self.pv.fit(train_boards, train_return, epochs=200,)
    
     
  def save(self):
    self.pv.save("/content/drive/MyDrive/Colab/pv_network_0927")


In [13]:
dn = DNN(Game(5))
dn.pv.summary()

Model: "sequential_88"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_88 (Flatten)        (None, 25)                0         
                                                                 
 dense_176 (Dense)           (None, 512)               13312     
                                                                 
 dropout_88 (Dropout)        (None, 512)               0         
                                                                 
 dense_177 (Dense)           (None, 26)                13338     
                                                                 
Total params: 26,650
Trainable params: 26,650
Non-trainable params: 0
_________________________________________________________________


In [14]:
#dn.train_pv()

In [15]:
%%script false --no-raise-error

boards = dn.b
p = dn.pv.predict(numpy.array(boards))
for i in range(len(boards)):
  print(i)
  for j in range(5):
    print(boards[i][j], p[i][j*5:j*5+5])

In [16]:
###dn.save()

# GUI

In [17]:
class Go():


  def __init__(self, size):
    self.size = size
    scale = self.size/9
    if scale>1.2: scale = 1.2
    self.board_size = 600*scale
    if self.board_size<200: self.board_size=200
    self.board_offset = 40
    self.board_margin = 40
    self.grid_x_low = self.board_margin+self.board_offset
    self.grid_x_high = self.board_size-self.board_margin
    self.grid_y_low = self.board_size-self.board_margin-self.board_offset
    self.grid_y_high = self.board_margin
    self.board_grid = (self.board_size-self.board_offset-2*self.board_margin)/(self.size-1)

    self.game = Game(self.size)
    self.black = Player(1,self.game)
    self.white = Player(-1,self.game)
    self.current_move = []
    self.canvas = Canvas()
    h1,h2,h3,h4,h5,self.output = self.control()
    display(self.canvas,h1,h2,h3,h4,h5,self.output)
    self.canvas_event()
    self.draw()

    self.max_game_step = 0
    self.max_mcts = 0
    self.max_rollout = 0
    self.read_history()
    #self.history = {} 

    return

  def draw(self):

    self.canvas.width = self.board_size
    self.canvas.height = self.board_size
    background_color = "#ffcc55"
    grid_color = "#223344"

    with hold_canvas(self.canvas):
      self.canvas.clear()
      self.canvas.fill_style = background_color
      self.canvas.stroke_style = grid_color
      self.canvas.fill_rect(0, 0, self.canvas.width, self.canvas.height)

      for x in range(self.size):
        gx = x*self.board_grid+self.board_offset+self.board_margin
        self.canvas.stroke_line(gx, self.grid_y_low, gx, self.grid_y_high)
      for y in range(self.size):
        gy = self.board_size-y*self.board_grid-self.board_offset-self.board_margin
        self.canvas.stroke_line(self.grid_x_low, gy, self.grid_x_high, gy)


      self.canvas.fill_style = "black"
      self.canvas.font = "24px serif"
      for x in range(self.size):
        self.canvas.stroke_text(x, x*self.board_grid+self.grid_x_low, self.grid_y_low+self.board_offset, max_width=30)
      for y in range(self.size):
        self.canvas.stroke_text(y, self.grid_x_low-self.board_offset, self.grid_y_low-y*self.board_grid)

      for x in range(self.size):
          for y in range(self.size):
            if self.game.show_board[x][y] == 1:
              self.canvas.fill_style = "black"
              self.canvas.fill_circle(x*self.board_grid+self.grid_x_low,self.grid_y_low-y*self.board_grid,self.board_grid/3)
              self.canvas.fill_style = "gray"
              self.canvas.stroke_circle(x*self.board_grid+self.grid_x_low,self.grid_y_low-y*self.board_grid,self.board_grid/3)
            elif self.game.show_board[x][y] == -1:
              self.canvas.fill_style = "white"
              self.canvas.fill_circle(x*self.board_grid+self.grid_x_low,self.grid_y_low-y*self.board_grid,self.board_grid/3)
              self.canvas.fill_style = "gray"
              self.canvas.stroke_circle(x*self.board_grid+self.grid_x_low,self.grid_y_low-y*self.board_grid,self.board_grid/3)
              self.canvas.fill_style = "white"
      m = self.game.history["move"][self.game.step]
      if isinstance(m,list) and isinstance(m[0],int):
        current_move = [m[0], m[1]]
        self.canvas.fill_style = "red"
        x,y = current_move[0],current_move[1]
        self.canvas.fill_circle(x*self.board_grid+self.grid_x_low,self.grid_y_low-y*self.board_grid,self.board_grid/6)
    return self.canvas
  
  def canvas_event(self):
    out = Output()

    @out.capture()
    def handle_mouse_move(x, y):
      return

    @out.capture()
    def handle_mouse_down(x, y):
      gx = round((x-self.grid_x_low)/self.board_grid)
      gy = round((self.grid_y_low-y)/self.board_grid)
      moves = self.game.moves()
      self.game.make_move([gx,gy])
      self.current_move = [gx,gy]
      self.draw()
      with self.output:
        self.output.clear_output()
        print("Touch-> Step: ",self.game.step, ", ", self.current_move) 

    self.canvas.on_mouse_move(handle_mouse_move)
    self.canvas.on_mouse_down(handle_mouse_down)
    return



  def control(self):
    button1 = widgets.Button(description=">")
    button2 = widgets.Button(description="<")
    button5 = widgets.Button(description=">>")
    button6 = widgets.Button(description="<<")
    button7 = widgets.Button(description=">|")
    button8 = widgets.Button(description="|<")
    button3 = widgets.Button(description="MCTS")
    button10 = widgets.Button(description="Undo")
    button9 = widgets.Button(description="Run")
    button13 = widgets.Button(description="Run100")
    button4 = widgets.Button(description="New Game")
    button11 = widgets.Button(description="Pass")
    button12 = widgets.Button(description="DNN")
    button14 = widgets.Button(description="-")
    button15 = widgets.Button(description="+")
    self.hisSlider = widgets.IntSlider(
        value=7,
        min=0,
        max=10,
        step=1,
        description='',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    def f(n):
      try:
        if self.history:
          keys = list(self.history.keys())
          if n<len(keys):
            print(time.ctime(float(keys[n])), self.history[keys[n]]["result"])
            self.game.history=self.history[keys[n]]
            on_button7_clicked(None)
      except AttributeError:
        print("no history")
      self.draw()
    t = widgets.interactive_output(f, {'n': self.hisSlider})

    output = widgets.Output()

    def game_display(message):
      self.draw()
      output.clear_output()
      #clear_output(wait=True) 
      with output:
        print(message)
        if self.game.state == "Over":
          print("game over - ", self.game.history["result"])
      
    h1 = HBox([button8, button6, button2, button1, button5, button7])
    h2 = HBox([button11, button10, button4])
    h3 = HBox([button3,button9, button13])
    h4 = HBox([button12])
    h5 = HBox([button14,self.hisSlider,button15, t])

    game_display("New Game")


    def on_button1_clicked(b):
      self.game.forward(1)
      game_display(f'> Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button2_clicked(b):
      self.game.backward(1)
      game_display(f'< Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button5_clicked(b):
      self.game.forward(10)
      game_display(f'>> Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button6_clicked(b):
      self.game.backward(10)
      game_display(f'<< Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button7_clicked(b):
      self.game.forward(0)
      game_display(f'>| Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button8_clicked(b):
      self.game.backward(0)
      game_display(f'|< Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button3_clicked(b):
      if self.game.state == "Over":
        game_display("Game is over! Start a new game.")
        return
      self.game.forward(0)
      game_display("MCTS searching...")
      if self.game.turn==1:
        self.black.mcts()
        if self.black.mc.max_game_step > self.max_game_step: self.max_game_step = self.black.mc.max_game_step
        if self.black.mc.max_mcts > self.max_mcts: self.max_mcts = self.black.mc.max_mcts
        if self.black.mc.max_rollout > self.max_rollout: self.max_rollout = self.black.mc.max_rollout
      elif self.game.turn==-1:
        self.white.mcts()
        if self.white.mc.max_game_step > self.max_game_step: self.max_game_step = self.white.mc.max_game_step
        if self.white.mc.max_mcts > self.max_mcts: self.max_mcts = self.white.mc.max_mcts
        if self.white.mc.max_rollout > self.max_rollout: self.max_rollout = self.white.mc.max_rollout
      if self.game.state == "Over":
        s = len(self.game.history["move"])-1
        game_display(f'MCTS-> Step: {s} , {self.game.history["move"][-1]} - Game Over! Winner: {self.game.check_Win()}')
        self.write_history()
      else:
        game_display(f'MCTS-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')

    def on_button10_clicked(b):
      self.game.unmake_move()
      game_display(f'Undo-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')

    def on_button9_clicked(b):
      if self.game.state == "Over":
        game_display("Game is over! Start a new game.")
        return
      self.game.forward(0)
      game_display("Running...")
      steps = 2* self.game.size*self.game.size
      while self.game.state == "Proceeding" and steps>0:
        steps -=1
        if self.game.turn==1:
          self.black.mcts()
          if self.black.mc.max_game_step > self.max_game_step: self.max_game_step = self.black.mc.max_game_step
          if self.black.mc.max_mcts > self.max_mcts: self.max_mcts = self.black.mc.max_mcts
          if self.black.mc.max_rollout > self.max_rollout: self.max_rollout = self.black.mc.max_rollout

        elif self.game.turn==-1:
          self.white.mcts()
          if self.white.mc.max_game_step > self.max_game_step: self.max_game_step = self.white.mc.max_game_step
          if self.white.mc.max_mcts > self.max_mcts: self.max_mcts = self.white.mc.max_mcts
          if self.white.mc.max_rollout > self.max_rollout: self.max_rollout = self.white.mc.max_rollout

        if self.game.state == "Over":
          s = len(self.game.history["move"])-1
          winner = self.game.check_Win()
          game_display(f'Run-> Step: {s} , {self.game.history["move"][-1]} - Game Over! Winner: {winner}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')
          self.write_history()
        else:
          game_display(f'Run-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')

    def on_button4_clicked(b):
      self.game = Game(self.size)
      self.black = Player(1,self.game)
      self.white = Player(-1,self.game)
      game_display(f'New Game')

    def on_button11_clicked(b):
      if self.game.state == "Over":
        game_display("Game is over! Start a new game.")
        return
      self.game.forward(0)
      game_display("thinking...")
      self.game.make_move("Pass")
      if self.game.state == "Over":
        s = len(self.game.history["move"])-1
        game_display(f'Play-> Step: {s} , {self.game.history["move"][-1]} - Game Over! Winner: {self.game.check_Win()}')
      else:
        game_display(f'Play-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button12_clicked(b):
      if self.game.state == "Over":
        game_display("Game is over! Start a new game.")
        return
      self.game.forward(0)
      game_display("DNN thinking...")
      if self.game.turn==1:
        self.black.dnn()
      elif self.game.turn==-1:
        self.white.dnn()
      if self.game.state == "Over":
        s = len(self.game.history["move"])-1
        game_display(f'DNN-> Step: {s} , {self.game.history["move"][-1]} - Game Over! Winner: {self.game.check_Win()}')
      else:
        game_display(f'DNN-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}')

    def on_button13_clicked(b):
      for i in range(100):
        self.game = Game(self.size)
        self.black = Player(1,self.game)
        self.white = Player(-1,self.game)
        game_display(f'Running Game {i}')

        while self.game.state == "Proceeding":
          if self.game.turn==1:
            self.black.mcts()
            if self.black.mc.max_game_step > self.max_game_step: self.max_game_step = self.black.mc.max_game_step
            if self.black.mc.max_mcts > self.max_mcts: self.max_mcts = self.black.mc.max_mcts
            if self.black.mc.max_rollout > self.max_rollout: self.max_rollout = self.black.mc.max_rollout

          elif self.game.turn==-1:
            self.white.mcts()
            if self.white.mc.max_game_step > self.max_game_step: self.max_game_step = self.white.mc.max_game_step
            if self.white.mc.max_mcts > self.max_mcts: self.max_mcts = self.white.mc.max_mcts
            if self.white.mc.max_rollout > self.max_rollout: self.max_rollout = self.white.mc.max_rollout

          if self.game.state == "Over":
            s = len(self.game.history["move"])-1
            winner = self.game.check_Win()
            game_display(f'Run {i}-> Step: {s} , {self.game.history["move"][-1]} - Game Over! Winner: {winner}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')
            self.write_history()
          else:
            game_display(f'Run {i}-> Step: {self.game.step} , {self.game.history["move"][self.game.step]}, {self.max_game_step}/{self.max_mcts}/{self.max_rollout}')

    def on_button14_clicked(b):
      self.hisSlider.value -= 1

    def on_button15_clicked(b):
      self.hisSlider.value +=1

    button1.on_click(on_button1_clicked)
    button2.on_click(on_button2_clicked)
    button5.on_click(on_button5_clicked)
    button6.on_click(on_button6_clicked)
    button7.on_click(on_button7_clicked)
    button8.on_click(on_button8_clicked)
    button3.on_click(on_button3_clicked)
    button10.on_click(on_button10_clicked)
    button9.on_click(on_button9_clicked)
    button4.on_click(on_button4_clicked)
    button11.on_click(on_button11_clicked)
    button12.on_click(on_button12_clicked)
    button13.on_click(on_button13_clicked)
    button14.on_click(on_button14_clicked)
    button15.on_click(on_button15_clicked)

    return h1, h2, h3, h4, h5, output

  def write_history(self, filename='/content/drive/MyDrive/Colab/history_20221002.json'):
    if not exists(filename):
      h = {}
      with open(filename, "w") as fp:
        json.dump(h, fp)
    with open(filename, 'r+') as fp:
      self.history = json.load(fp)
      self.history[time.time()] = self.game.history
      # Sets file's current position at offset.
      fp.seek(0)
      # convert back to json.
      json.dump(self.history, fp, indent = 4)

    if len(self.history)>0:
      self.hisSlider.min = 0
      self.hisSlider.max = len(self.history)-1
      if self.hisSlider.max < 1: self.hisSlider.max=1
      self.hisSlider.value = len(self.history)-1
    else:
      self.hisSlider.min = 0
      self.hisSlider.max = 1
      self.hisSlider.value=0

  def read_history(self, filename='/content/drive/MyDrive/Colab/history_20221002.json'):
    if exists(filename):
      with open(filename, 'r') as fp:
        self.history=json.load(fp)
    else:
      self.history = {}

    if len(self.history)>0:
      self.hisSlider.min = 0
      self.hisSlider.max = len(self.history)-1
      if self.hisSlider.max<1: self.hisSlider.max=1
      self.hisSlider.value = len(self.history)-1
    else:
      self.hisSlider.min = 0
      self.hisSlider.max = 1
      self.hisSlider.value=0
    #  assert False, "history file not exist"

  def evaluate_history(self):
    for key in self.history.keys():
      self.game.board = self.history[key]["board"][-1]
      black, white = self.game.check_Land()
      winner = self.game.check_Win()
      self.history[key]["result"] = [winner, black, white]
      print(time.ctime(float(key)),winner, black, white)




# playground

In [23]:
go = Go(5)

Canvas(height=333, width=333)

Output()

In [19]:
go.game.history

{'move': [None],
 'board': [[[0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0]]]}

In [20]:
go.game.group_[4][1]

In [21]:
go.game.group_[1][0]

In [22]:
assert False
class Gnode():
  def __init__(self, game):
    self.game=game
    self.leaf =True
    self.terminal=False
  def expand(self):
    self.moves = deepcopy(self.game.moves())
    self.children=[Gnode(self.game) for i in range(len(self.moves))]
    

class gametree():
  def __init__(self):
    self.game = Game(5)
    self.blackWin=0
    self.whiteWin=0
    self.draw=0
    self.other=0
    self.root = Gnode(self.game)

  def count(self, node):
    global n
    if node.terminal: return
    node.expand()
    for i in range(len(node.moves)):
      self.game.make_move(node.moves[i])
      if self.game.state == "Over":
        winner = self.game.check_Win()
        if winner == 1:
          self.blackWin +=1
        elif winner == -1:
          self.whiteWin += 1
        elif winner == 0:
          self.draw +=1
        else:
          self.other+=1

        if False:#self.blackWin % 10000 == 0:
          go.history[time.time()] = deepcopy(self.game.history)
          go.hisSlider.max = len(go.history)-1
          #go.write_history()
          if n>=20:
            assert False, "abort"
          else:
            n += 1
            
        clear_output(wait=True)
        print(self.blackWin, self.whiteWin, self.draw,self.other,self.game.step,len(node.moves))
        node.children[i].terminal=True
      else:
        self.count(node.children[i])

      self.game.unmake_move()

    node.terminal =True
    node.children =[]

n=0
g=gametree()
root = Gnode(g.game)
g.count(root)
print("finished")


AssertionError: ignored

In [ ]:
for k in go.history.keys():
  print(time.ctime(float(k)),go.history[k]["move"])

In [ ]:
go.history

# Remark

To check:

Wed Sep 21 14:53:08 2022 [1, 13, 11]

Wed Sep 21 10:01:50 2022 [-1, 12, 13]

Wed Sep 21 08:56:30 2022 [1, 13, 10]

Tue Sep 20 11:02:51 2022 [1, 12, 11]

Tue Sep 20 10:37:05 2022 [1, 13, 11]

Tue Sep 20 07:14:57 2022 [1, 14, 11]

Mon Sep 19 05:03:25 2022 [1, 14, 7]

Mon Sep 19 04:42:50 2022 [1, 12, 11]

Mon Sep 19 03:59:57 2022 [-1, 12, 13]

0921 MCTS iteration 500->1000

To Do:

A method to judge game result as early as possible and trim the redundant sub-trees.
long game doesn't mean Draw

Definitions for eye and incomplete eye

New eye not added to the group at last step. -solved

Recover game state, e.g. game.turn, when playing/undoing historical game -solved

Stacking the grouping instead of re-grouping when undoing the move
